In [ ]:
using SparseArrays
using LinearAlgebra
using NLopt
using PyPlot
using KrylovKit
using Statistics
using FiniteDifferences
using Revise
using IJulia
using TopologyOptimizationHelper

In [ ]:
Lx = 5
Ly = 5
res = 50
ε = ones(res * (Ly + 1), res * (Lx + 1))
ω = 2π

A, x, y = Maxwell2d(Lx, Ly, ε, ω; resolution=res)
N, M = size(ε)
b = zeros(N, M)
b[N÷2, M÷2] = 1;

In [ ]:
function ring_resonator(X, inner, outer; resolution=20)
    # Get matrix dimensions
    N, M = size(X)
    
    # Compute the center of the matrix
    center_y = N÷2
    center_x = M÷2
    
    # Assign value to entries within the given radii
    for i in 1:M
        for j in 1:N
            # Calculate the distance from the center, considering the resolution
            square_dist = ((i - center_x) / res)^2 + ((j - center_y) / res)^2
            if inner^2 <= square_dist <= outer^2
                X[j, i] = 12
            end
        end
    end
    return X
end

In [ ]:
function quarter_wave_2d(X, Lx, Ly, n; λ=1)
    # Get matrix dimensions
    N, M = size(X)
    
    # Compute the center of the matrix
    center_y = N÷2
    center_x = M÷2
    d1 = λ/4
    d12 = d1/sqrt(12)
    period = d1 + d12
    
    # Assign value to entries within the given radii
    for k in 0:n
        s_k = λ/4 + k * period
        #s_k < abs(x - L/2) <= s_k + d12
        for i in 1:M
            for j in 1:N
                # Calculate the distance from the center, considering the resolution
                square_dist = ((center_x - i) / res)^2 + ((center_y - j) / res)^2
                if square_dist > (Lx^2 + Ly^2) / 4 || square_dist < λ^2/16
                    X[j,i] = 1
                elseif s_k^2 < square_dist <= (s_k + d12)^2
                    X[j,i] = 12
                end
            end
        end
    end

    return X
end

In [ ]:
function quarter_wave_pointwise(x, y, Lx, Ly, n; λ=1, resolution=res, dpml=0.5)
    center_x = (Lx + 2dpml) /  2
    center_y = (Ly + 2dpml) / 2

    square_dist = (x - center_x)^2 + (y - center_y)^2
    if x^2 + y^2 > (Lx^2 + Ly^2) / 4 || square_dist < λ^2/16
        return 1 
    end

    d1 = λ/4
    d12 = d1/sqrt(12)
    period = d1 + d12

    for k in 0:n
        s_k = λ/4 + k * period

        if s_k^2 < square_dist <= (s_k + d12)^2
            return 12
        end
    end

    return 1
end

In [ ]:
X = ones(N, M)
idx_x = 0
idx_y = 0
for hor in x
    idx_x = idx_x + 1
    idx_y = 0
    for vert in y
        idx_y = idx_y + 1
        X[idx_y, idx_x] = quarter_wave_pointwise(hor, vert, Lx, Ly, 4)
    end
end

In [ ]:
imshow(X, cmap="coolwarm", vmin=1, vmax=12)

In [ ]:
rings_optimum_2d = quarter_wave_2d(ones(N,M), Lx, Ly, 10);

In [ ]:
imshow(rings_optimum_2d, cmap="coolwarm", vmin=1, vmax=12)

In [ ]:
A_opt, _, _ = Maxwell2d(Lx, Ly, rings_optimum_2d, ω; resolution=res)
rings_opt = sqrt(Arnoldi_eig(A_opt, vec(rings_optimum_2d), ω, vec(b))[1])
Q_opt = -real(rings_opt) / 2imag(rings_opt)

In [ ]:
E⁻¹ = spdiagm(1 ./ vec(rings_optimum_2d))
A, x, y = Maxwell2d(Lx, Ly, rings_optimum_2d, ω; resolution=res)
vals, vecs, info = eigsolve(z -> E⁻¹ * A * z + ω^2 .* z, vec(b), 1, EigSorter(λ -> abs(λ - ω^2); rev = false), Arnoldi()) 
u₀ = vecs[1];

In [ ]:
imshow(reshape(abs.(u₀), N,M), cmap="coolwarm", vmin=0)
colorbar(label=L"|u₀|")

In [ ]:
function mod_LDOS_Optimize2d(Lx, Ly, ε, ω, b, x₀; dpml=0.5, resolution=20, Rpml=1e-20, ftol=1e-4, max_eval=500, design_dimensions=(Lx,Ly))
    A, x, y = Maxwell2d(Lx, Ly, ε, ω; dpml, resolution, Rpml)
    ε = vec(ε)
    b = vec(b)
    D² = A + ω^2 .* spdiagm(ε)
    M, N = length(x), length(y)
    mod_LDOS_vals = Float64[]
    mod_omegas = ComplexF64[]
    
    function mod_LDOS_obj(ε, grad)
        ε = vec(ε)
        E = spdiagm(ε)
        A = D² - ω^2 .* E
        E⁻¹ = spdiagm(1 ./ ε)
        C = E⁻¹ * A
        LU = lu(C)
        vals, vecs, _ = eigsolve(z -> LU \ z, x₀, 1, :LM, Arnoldi())
        vals = sqrt.(1 ./ vals .+ ω^2)
        ω₀, u₀ = vals[1], vecs[1]
        A₀ = D² - real(ω₀)^2 .* E
        v = A₀ \ b
        w = A₀' \ b
        
        LDOS = -imag(v' * b)
        
        reals = real.(vals)
        imaginaries = imag.(vals)
        if !isempty(grad) 
            ∂LDOS_∂ε = -imag.(real(ω₀)^2 .* conj.(v) .* w)
            ∂LDOS_∂ω = -imag(2real(ω₀) .* v' * E * w)
            ∂ω_∂ε = -ω₀ .* u₀.^2 ./ 2sum(u₀.^2 .* ε)
            ∇LDOS = ∂LDOS_∂ε .+  ∂LDOS_∂ω .* real.(∂ω_∂ε)

            grad .= ∇LDOS
        end

        push!(mod_LDOS_vals, LDOS)
        push!(mod_omegas, ω₀)

        IJulia.clear_output(true)  # Clear notebook output
        plt.clf()  # Clear the current figure, removes all previous plots
        
        fig, ax = plt.subplots(1,2, figsize=(15, 5))  # Fix the figure size to avoid shrinking
        ε_graph = ax[1].imshow(reshape(ε, N,M), cmap="coolwarm", vmin=1, vmax=12)
        ax[1].set_title("Iteration $(length(mod_LDOS_vals)), LDOS = $LDOS")
        plt.colorbar(ε_graph, label=L"ε_{opt}", ax=ax[1])

        ax[2].scatter(reals[2:end], imaginaries[2:end], color="blue")
        ax[2].scatter(reals[1], imaginaries[1], color="red")
        ax[2].scatter(real(ω), imag(ω), color="green")
        ax[2].set_xlim(6,6.5)
        ax[2].set_ylim(-0.5, 0.01)

        display(gcf())  # Display the current figure in the notebook
        sleep(0.1)  # Pause to visualize the update

        return LDOS
    end

    function freq_constraint(ε, grad)
        ω₀, ∂ω_∂ε = Eigengradient(A, ε, ω, x₀)
        if !isempty(grad) 
            grad .= -real.(∂ω_∂ε)
        end

        return ω - real(ω₀)
    end

    design_x, design_y = design_dimensions
    x_indices = -design_x / 2 .< x .- mean(x) .< design_x / 2
    y_indices = -design_y / 2 .< y .- mean(y) .< design_y / 2
    ub = ones(N,M)
    ub[y_indices, x_indices] .= 12
    ub = vec(ub)
    
    opt = Opt(:LD_CCSAQ, M * N)
    # opt.params["verbosity"] = 1
    opt.lower_bounds = 1
    opt.upper_bounds = ub
    opt.ftol_rel = ftol
    opt.maxeval = max_eval
    opt.max_objective = mod_LDOS_obj
    opt.initial_step = 1e-3
    inequality_constraint!(opt, freq_constraint)

    (mod_LDOS_opt, mod_ε_opt, ret) = optimize(opt, ε)
    A_opt, _, _ = Maxwell2d(Lx, Ly, mod_ε_opt, ω; resolution=res)
    ω₀_opt = sqrt(Arnoldi_eig(A_opt, vec(mod_ε_opt), ω, vec(b))[1])
    Q_opt = -real(ω₀_opt) / 2imag(ω₀_opt)
    
    IJulia.clear_output(true)  # Clear notebook output
    plt.clf()

    @show numevals = opt.numevals # the number of function evaluations
    @show ω₀_opt
    @show Q_opt
    
    return mod_LDOS_opt, reshape(mod_ε_opt, N,M), mod_LDOS_vals, mod_omegas, x, y
end

In [ ]:
mod_LDOS_opt, mod_ε_opt, mod_LDOS_vals, mod_omegas, _, _ = mod_LDOS_Optimize2d(Lx, Ly, rings_optimum_2d, ω, b, vec(b); resolution=res, max_eval=100, ftol=0)
@show mod_LDOS_opt
imshow(mod_ε_opt, cmap="coolwarm", vmin=1, vmax=12)
colorbar(label=L"ε_{opt}")